In [3]:
import math
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
from sys import getsizeof
import sys

import pickle
from operator import itemgetter

In [2]:
#get the behavioral data and the list of subjects allocated for training
wtpw1_behavdesign_clean = pd.read_csv("../data/wtpw1_behavdesign_clean.csv")
test_train_df = pd.read_csv("../data/train_test_markers_20210601T183243.csv")
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()

Based on `load_multisubject_raw_data` and `extract_raw_data.py`, this time we just extract finger press data.

### Example subject

In [8]:
from dev_wtp_io_utils import *

For a given subject label, wave, and run, we can load their WTP file....




In [5]:
print("extracting raw data")
sys.stdout.flush()

#get the behavioral data and the list of subjects allocated for training
wtpw1_behavdesign_clean = pd.read_csv("../data/wtpw1_behavdesign_clean.csv")
test_train_df = pd.read_csv("../data/train_test_markers_20210601T183243.csv")
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()

example_subj = train_subjs[0]
print(example_subj)



extracting raw data
DEV001


This time we'll need the raw `event_stage` data, not just the event data.

In [13]:
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)


subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

We've already processed the filtered run_event_df_list into wtpw1_behavdesign_clean, so we will match off that

In [14]:
del run_event_df_list #delete in order to avoid interference

In [38]:
event_data_to_use = wtpw1_behavdesign_clean.loc[
    :,['event_id','run','wave','subject','liking_rating','response']
                                                ]
event_stage_df_clean = event_data_to_use.merge(
         run_event_stage_df_list,
    how='left')   



In [39]:
events_choice_period = event_stage_df_clean[event_stage_df_clean.stage=='bid_prechoice']

That was straightforward!

In [40]:
training_Brain_Data = dev_wtp_io_utils.get_event_related_Brain_Data_for_all_subs_all_runs_fast([example_subj],1,events_choice_period)

DEV001 (1 2 3 4 )
extracted all data. concatenating...
...concatenated.
combining into a Brain_Data file....
...done.


#### Full code 

In [41]:
import math
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
from operator import itemgetter
import dev_wtp_io_utils

#get the cleaned behavioral data and the list of subjects allocated for training
wtpw1_behavdesign_clean = pd.read_csv("../data/wtpw1_behavdesign_clean.csv")
test_train_df = pd.read_csv("../data/train_test_markers_20210601T183243.csv")
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()

#that data doesn't include the event_stage data, so let's get that and merge it in.
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)
del run_event_df_list
event_data_to_use = wtpw1_behavdesign_clean.loc[
    :,['event_id','run','wave','subject','liking_rating','response']
                                                ]
event_stage_df_clean = event_data_to_use.merge(
         run_event_stage_df_list,
    how='left')   


#now run the code specifically to grab the choice events
events_choice_period = event_stage_df_clean[event_stage_df_clean.stage=='bid_prechoice']



training_Brain_Data_5 = dev_wtp_io_utils.get_event_related_Brain_Data_for_all_subs_all_runs_fast(train_subjs[0:5],1,events_choice_period)

with open('../data/Brain_Data_raw_fingerpress_5subs.pkl', 'wb') as pkl_file:
    pickle.dump(training_Brain_Data_5,pkl_file)

subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

### Crude HRF adjustment

In [43]:
import math
import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
from operator import itemgetter
import dev_wtp_io_utils

#get the cleaned behavioral data and the list of subjects allocated for training
wtpw1_behavdesign_clean = pd.read_csv("../data/wtpw1_behavdesign_clean.csv")
test_train_df = pd.read_csv("../data/train_test_markers_20210601T183243.csv")
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()

#that data doesn't include the event_stage data, so let's get that and merge it in.
behavioral_data_path = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/Tasks/WTP/output/'
[run_list, run_event_df_list, run_event_stage_df_list] = dev_wtp_io_utils.get_comprehensive_run_data_from_mat_dir(behavioral_data_path)
del run_event_df_list
event_data_to_use = wtpw1_behavdesign_clean.loc[
    :,['event_id','run','wave','subject','liking_rating','response']
                                                ]
event_stage_df_clean = event_data_to_use.merge(
         run_event_stage_df_list,
    how='left')   


#now run the code specifically to grab the choice events
events_choice_period = event_stage_df_clean[event_stage_df_clean.stage=='bid_prechoice'].copy()

#now we do a crude, hacky HRF adjustement. Just add 5 seconds to the choice period.
events_choice_period.onset = events_choice_period.onset + 5
events_choice_period.end = events_choice_period.end + 5

training_Brain_Data_5 = dev_wtp_io_utils.get_event_related_Brain_Data_for_all_subs_all_runs_fast(train_subjs[0:5],1,events_choice_period)

with open('../data/Brain_Data_raw_fingerpress_5subs.pkl', 'wb') as pkl_file:
    pickle.dump(training_Brain_Data_5,pkl_file)

subject 040, subject 039, subject 168, subject 063, subject 129, subject 067, subject 150, subject 159, 
SUBJ 159, 2, run14
no data for this run.
subject 004, subject 091, subject 180, subject 008, subject 095, subject 099, subject 032, 
SUBJ 032, 2, run2
no data for this run.
subject 036, subject 171, subject 100, 
SUBJ 100, 3, run2
no data for this run.
subject 109, subject 041, subject 060, subject 059, subject 130, subject 064, subject 068, subject 001, subject 087, subject 192, subject 005, subject 121, subject 999, 
SUBJ 999, 2, run1
no data for this run.


SUBJ 999, 2, run2
Data for this run is not in the expected format or is missing. Skipping this run.
time: 29-Oct-2020 17:09:09

StartTime: 3035.874545661

Jitter: array of length 16

TrialStart: array of length 2

ISI: array of length 2

FoodOn: array of length 2

BidOn: array of length 2

FoodOnset: array of length 2

BidOnset: array of length 2

FoodDuration: array of length 2

BidDuration: array of length 2

FoodPic: array 

In [46]:
events_choice_period = event_stage_df_clean[event_stage_df_clean.stage=='bid_prechoice'].copy()

#now we do a crude, hacky HRF adjustement. Just add 5 seconds to the choice period.
events_choice_period.onset = events_choice_period.onset + 5
events_choice_period.end = events_choice_period.end + 5


In [44]:
events_choice_period.sample(6)

,event_id,run,wave,subject,liking_rating,response,stage,onset,end
13439,10,run2,1,DEV033,1,7.0,bid_prechoice,110.415817,111.420235
2298,5,run1,1,DEV058,4,8.0,bid_prechoice,55.974459,56.438621
26609,2,run4,1,DEV105,4,8.0,bid_prechoice,22.708009,23.520676
7125,14,run1,1,DEV166,2,7.0,bid_prechoice,151.875630,152.547169
33396,15,run4,1,DEV047,4,8.0,bid_prechoice,161.688848,163.533345
27537,4,run4,1,DEV075,1,5.0,bid_prechoice,43.716022,44.481967
